In [1]:
!pip install transformers



In [2]:
pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [7]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import Audio, load_dataset
import jiwer

# Load the model and processor
processor = WhisperProcessor.from_pretrained("openai/whisper-base")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-base")
forced_decoder_ids = processor.get_decoder_prompt_ids(language="tamil", task="transcribe")

# Load the streaming dataset
ds = load_dataset("common_voice", "ta", split="test", streaming=True)
ds = ds.cast_column("audio", Audio(sampling_rate=16_000))

transcriptions = []

# Iterate over the dataset and transcribe each sample
for sample in ds:
    input_speech = sample["audio"]
    input_features = processor(input_speech["array"], sampling_rate=input_speech["sampling_rate"], return_tensors="pt").input_features

    # Generate token ids
    predicted_ids = model.generate(input_features, forced_decoder_ids=forced_decoder_ids)
    # Decode token ids to text
    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
    
    transcriptions.append(transcription[0])

# Calculate Word Error Rate (WER)
references = [sample["sentence"] for sample in ds]
wer = jiwer.wer(references, transcriptions)
print("WER:", wer)


Got disconnected from remote data host. Retrying in 5sec [1/20]
Got disconnected from remote data host. Retrying in 5sec [1/20]
Got disconnected from remote data host. Retrying in 5sec [1/20]


WER: 1.201178566534192


In [16]:
# Create a dataframe to display references and transcriptions
import pandas as pd
data = {
    "Reference": references,
    "Transcription": transcriptions
}
df = pd.DataFrame(data)

# Display the dataframe
display(df)

,Reference,Transcription
0,அன்றுநீர் சொன்னபடி அவ்விரண்டு மூலிகையைச்,அந்த நீர் சொன்னப்படி அவ்விரண்டுப் புரிக்கயே
1,என்பகைவன்; உன்னாசை என்றன் ஆசை!,என்பாகய்வன் உண்ணாசை என்று நாசை
2,தொங்கும் நரம்பின் தூளே! இதைக்கேள்:,தொங்கும் நரம்பின் துளே இதைக்கிள்
3,சிவனென்றும் அரியென்றும் சித்தார்த்த னென்றும்,சிவனென்றும் அரிென்றும் சிடாட்டனென்றும்
4,முற்றும் அறிந்திடக் கூடுமென்றே - அவள்,உத்தும் அரிந்திடக் குடுமேன் ஜே அவள்
...,...,...
1776,இவன் படிந்து வேலை செய்வான் என்ற நம்பிக்கை எனக்...,இவன் படிந்து வலை சேவான என்று நம்பிக்கே எனக்கு...
1777,கொண்டுபோகப்பட்ட தென்றும் பிரித்தானியக் கலைக்கள...,பண்டு போகப் பட்டத்து என்றும் குற்றானியைக் கலை...
1778,"காட்டில் போய் யாருக்கும் தெரியாமல், மறைந்தொழுக...",ஆட்கில் போயார்க்கும் தெரியாமல் மரைந்தோடுகிறேன...
1779,லஞ்சம் வாங்குபவர்களைத் தண்டிப்பதைவிடக் கொடுப்ப...,لنجம் வாங்கபவற்கலை இதன் இப்பதை விடுகிறது குடு...


In [14]:
(references)

['அன்றுநீர் சொன்னபடி அவ்விரண்டு மூலிகையைச்',
 'என்பகைவன்; உன்னாசை என்றன் ஆசை!',
 'தொங்கும் நரம்பின் தூளே! இதைக்கேள்:',
 'சிவனென்றும் அரியென்றும் சித்தார்த்த னென்றும்',
 'முற்றும் அறிந்திடக் கூடுமென்றே - அவள்',
 'தெளிவளிக்க இருட்கதவை உடைத்தெறிந்தான் பரிதி!',
 'நாடி வந்து நடுவிற் படுத்தாள்.',
 'பேசவொண் ணாதவர் ஊரினில் துஷ்டர்',
 'படமெடுத்தாடும்; தமிழர் பங்கமெலாம் போமே!',
 'முட்டைபோல் உதடுகள் முன்னே தோன்ற',
 'பெண்ணுக் கிதுதகுமோ? வண்ணமலர்ச் சோலையிலே',
 '"தோகையவள் ""என்காதல் துரையே கேளாய்!"',
 'கொள்ளை கொடுத்தனன் உள்ளத்தினை! - அவள்',
 '"""தெருவினிலேபனி"" என்றாள். ஆமென்று சொன்னேன்;"',
 'என்னைமுடி சூடுகென்றார். உங்கட் கெல்லாம்',
 '"""குவிந்த உன் உடற்சதையைப் - பல"',
 'என்று கூறி எழுந்து போனான்!இனிதாய்ப் பகல்மணி பனிரண் டானது',
 'அன்னையும் ஆசானும் ஆருயிரைக் காப்பானும்',
 'சாய்ந்த பாலை நக்கித் தன்தலை',
 'வந்த அண்ணன் வந்த வழியே',
 'சாதிஉயர் வென்றும், தனத்தால் உயர்வென்றும்',
 'காய்ச்சும் இரும்பிடை நீர்த்துளி ஆகக்',
 'பூச்சூடு கின்றார்; புறக்கணித் தார்எனை!',
 'எங்கள் அண்ணன் ஏறி வந்த',
 '"வேரோடு பேர்த்

In [18]:
transcriptions

[' அந்த நீர் சொன்னப்படி அவ்விரண்டுப் புரிக்கயே',
 ' என்பாகய்வன் உண்ணாசை என்று நாசை',
 ' தொங்கும் நரம்பின் துளே இதைக்கிள்',
 ' சிவனென்றும் அரிென்றும் சிடாட்டனென்றும்',
 ' உத்தும் அரிந்திடக் குடுமேன் ஜே அவள்',
 ' இளிவிற்க்கு இருட்டுக்கத்தை வைப்படைத் தெரிந்தான் பரிதி.',
 ' நாடி வந்து நடுவிர் படித்தால்',
 ' பேச வண்ணாதவர் உப்பிறி விள்ளுட்டும் துஷ்டர்',
 ' படமெடுத்தாடும் தமெடர் பங்கமெல்லாம் போமே',
 ' முட்டைபோலுக்கடுகள் உண்ணியது வந்தும் தார்.',
 ' பெண்க்கிடு தகுமோ வணமலர் சோலையிலே',
 ' تو گயமல் என் காதல் துரைய கேலாய்.',
 ' கொள்ளை கொடுத்தனான் உள்ளத்தினை அவள்',
 ' தெரிவினிலே பணி என்றால் ஆமண்டு சொன்னேன்.',
 ' என்னை முடிசுவுடுக்கு என்றார் உங்கள் கல்லாம்.',
 ' குவிந்த உன் உடன் சதையை பலா',
 ' என்று கூரி எழுந்து போனாக, கினிதாய் பலமணி பணிரண்டானது.',
 ' அண்ணயம் காசானும் ஆருகிறேன் காப்பானும்',
 ' சாய்ந்த பாலை நக்கிற் தந்தலை',
 ' பந்த அன்னன் பந்த வழியே',
 ' சாதி ஏறு வெண்டும் தனத்தால் ஏறு வெண்டும்',
 ' காஞ்சம் இரும்பிடை நீ துளியாக',
 ' புச்சுடுக்கின்றார் புரக்கனித்தார் என்னை?',
 ' என்கள் அன்னன் ஏறி வந்தா'

In [27]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import Audio, load_dataset
import jiwer

# Load the model and processor
processor = WhisperProcessor.from_pretrained("openai/whisper-base")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-base")
forced_decoder_ids = processor.get_decoder_prompt_ids(language="tamil", task="transcribe")

# Load the streaming dataset
ds = load_dataset("common_voice", "ta", split="test", streaming=True)
ds = ds.cast_column("audio", Audio(sampling_rate=16_000))

transcriptions = []

# Transliteration module
import tamil_unicode_map

class Mapper(object):
    def __init__(self, charmap):
        self.codepoint_to_english = {}
        self.codepoint_to_category = {}
        self.codepoint_to_char = {}
        self.populate_map(charmap)
        self.categories = {
             'consonants': 'consonants',
             'pulli': 'pulli',
             'dependent_vowels_two_part': 'vowels',
             'dependent_vowels_left': 'vowels',
             'various_signs': 'vowels',
             'independent_vowels': 'vowels',
             'dependent_vowels_right': 'vowels'
        }

    def populate_map(self, charmap):
        for category in charmap:
            codepoints = charmap[category]
            for codepoint in codepoints:
                self.codepoint_to_char[codepoint] = codepoints[codepoint][0]
                in_english = codepoints[codepoint][1]
                if isinstance(in_english, tuple):
                    self.codepoint_to_english[codepoint] = in_english[0]
                else:
                    self.codepoint_to_english[codepoint] = in_english
                self.codepoint_to_category[codepoint] = category

    def in_english(self, c):
        return self.codepoint_to_english.get(c, '')

    def char_type(self, c):
        sub_type = self.codepoint_to_category.get(c, '')
        parent_type = self.categories.get(sub_type, '')
        return parent_type, sub_type

class TamilTransliterator(object):
    def __init__(self, charmap):
        self.mapper = Mapper(charmap)

    # This is for just one word
    def to_english(self, text):
        text = self.preprocess(text)
        output = []
        l = len(text)

        for c in text:
            in_english = self.mapper.in_english(c)
            parent_type, sub_type = self.mapper.char_type(c)
            if parent_type == 'pulli':
                if output and output[-1] == 'n':
                    output.pop()  # na followed by pulli becomes n
            elif parent_type == 'vowels' and sub_type != 'independent_vowels':
                if output:
                    output.pop()
                output.extend(in_english)
            else:
                output.extend(in_english)
        return ''.join(output)

    def preprocess(self, text):
        return text

    # This is for the whole text
    def transliterate(self, text):
        words = text.split()
        transliterated_words = [self.to_english(w) for w in words]
        return ' '.join(transliterated_words).lower()

t = TamilTransliterator(tamil_unicode_map.charmap)

transliterations=[]

# Iterate over the dataset and transcribe each sample
for sample in ds:
    input_speech = sample["audio"]
    input_features = processor(input_speech["array"], sampling_rate=input_speech["sampling_rate"], return_tensors="pt").input_features

    # Generate token ids
    predicted_ids = model.generate(input_features, forced_decoder_ids=forced_decoder_ids)
    # Decode token ids to text
    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
    
    # Transliterate the transcription
    
    transliterated_text = t.transliterate(transcription)
    transliterations.append(transliterated_text)
    transcriptions.append(transcription)




Got disconnected from remote data host. Retrying in 5sec [1/20]
Got disconnected from remote data host. Retrying in 5sec [1/20]
Got disconnected from remote data host. Retrying in 5sec [2/20]
Got disconnected from remote data host. Retrying in 5sec [3/20]
Got disconnected from remote data host. Retrying in 5sec [4/20]
Got disconnected from remote data host. Retrying in 5sec [5/20]
Got disconnected from remote data host. Retrying in 5sec [6/20]
Got disconnected from remote data host. Retrying in 5sec [7/20]
Got disconnected from remote data host. Retrying in 5sec [8/20]
Got disconnected from remote data host. Retrying in 5sec [9/20]
Got disconnected from remote data host. Retrying in 5sec [10/20]
Got disconnected from remote data host. Retrying in 5sec [11/20]


In [28]:
# Calculate Word Error Rate (WER)
references = [sample["sentence"] for sample in ds]
wer = jiwer.wer(references, transcriptions)
print("WER:", wer)

WER: 1.201178566534192


In [30]:
transliterations

['anatha niira sonanapapadi avavirannadupa purikakayee',
 'enapaakayavana unnannaasai enarru naasai',
 'thongakuma naramapina thulee ithaikakila',
 'sivanenarruma arenarruma sidaadadanenarruma',
 'uthathuma arinathidaka kudumeena jee avala',
 'ilivirrakaku irudadukakathathai vaipapadaitha therinathaana parithi',
 'naadi vanathu naduvira padithathaala',
 'peesa vannannaathavara upapirri vilaludaduma thussadara',
 'padameduthathaaduma thamedara pangakamelalaama pomee',
 'mudadaipolukakadukala unnanniyathu vanathuma thaara',
 'pennakakidu thakumo vannamalara solaiyilee',
 ' yamala ena kaathala thuraiya keelaaya',
 'kolalai koduthathanaana ulalathathinai avala',
 'therivinilee panni enarraala aamannadu sonaneena',
 'enanai mudisuvudukaku enarraara ungakala kalalaama',
 'kuvinatha una udana sathaiyai palaa',
 'enarru kuuri elllunathu ponaaka kinithaaya palamanni pannirannadaanathu',
 'annannayama kaasaanuma aarukirreena kaapapaanuma',
 'saayanatha paalai nakakirra thanathalai',
 'panatha an